
# Make sure to edit the first cell with the appropriate path

In [ ]:
import sys, os

#change the path to the directory that ctmc.py is located in on your system
sys.path.append(os.path.expanduser('~/source/discrete_states/'))

In [ ]:
from ctmc import ContinuousTimeMarkovChain as MC
from ctmc import normal_generator, gamma_generator, uniform_generator, cyclic_generator, detailed_balance_generator, arrhenius_pump_generator

import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
#you can start by making a bespoke rate equation if you want. 
R = abs(np.empty((3,3)))
# we can set the rates by hand, to be DB
E_0 = 1
E_1 = 2
E_2 = 3
R[0,1] = np.exp(-E_1+E_0) 
R[1,0] = 1/R[0,1]
R[0,2] = np.exp(-E_2+E_0) 
R[2,0] = 1/R[0,2]
R[1,2] = np.exp(-E_2+E_1)
R[2,1] = np.exp(-E_2+E_1)

# and for fun, let's add something non DB
R[0,2] *= np.exp(2)

#the rate matrix looks like
print(R)

In [ ]:
# now, we can instantiate this into a CTMC object
machine_R = MC(R=R) 
# lets look at R again, now as an atrribute of machine_R
print(machine_R.R)
#you should see that the diagonals have been rectified to be a proper CTMC matrix, and the scale has been set so the 
# max output from any state is -1. this happend whenever we instantiate a new CTMC object automatically.
print(f'the scaling factor was {machine_R.scale}')


In [ ]:
# you can also make use of a generator_function to make a rate matrix that follows some particular sampling rules:

#make 1 machines with 10 states, each transition rate will be sampled from the same normal distribution
# the default is mu=0, sigma=1
#machine = MC(generator=normal_generator, S=20, N=1000)
#but they can also be passed as arguments
#machine = MC(generator=normal_generator, S=4, N=1, mu=10, sigma=5)


#make 1 machines with 5 states, generated using the "cyclic" generator fo transition rates. max_jumps means a state can only jump to next nearest neighbor state
# this should impose a geometry on the states
#machine = MC(generator=cyclic_generator, S=10, N=1, max_jump=3)


#make 1 machines with 3 states, generated using the arrhenious pump generator. Here, we select 3 transitions to be artifically pumped 
# beyond the normal DB transitions the arrheniosu equation gives. The pump strength is positive so as to be in the regime where ness and meps can be different
machine = MC(generator=arrhenius_pump_generator, S=7, N=1, n_pumps=20, pump_strength=3)


In [ ]:

# find and the NESS state
print('getting ness')
ness = machine.get_ness()
# finds and the minimum entropy producing state
print('getting meps')
meps = machine.get_meps()
# returns a uniform state
print('getting unif')
unif = machine.get_uniform()


In [ ]:
# for states, we can grab a variety of values, like the entropy produciton rate
print('epr:')
print(machine.get_epr(ness), machine.get_epr(meps), machine.get_epr(unif))
#activity (which is the sum of all (positive) transition rates)
print('activity:')
print(machine.get_activity(ness), machine.get_activity(meps), machine.get_activity(unif))
# or probability current (which is the total amount of probability flowing between states, note it is zero for the NESS state)
print('current:')
print(machine.get_prob_current(ness), machine.get_prob_current(meps), machine.get_prob_current(unif))

In [ ]:
print(machine.get_prob_current(ness), machine.get_prob_current(meps), machine.get_prob_current(unif))

In [ ]:
# returns a random state drawn from a dirichlet distrbution over all states
random = machine.get_random_state()

# returns a "localized" state which is basically a guassian centered on a particular state, putting the states on a ring, default is random peak and variance
random_local = machine.get_local_state()
# you can put in manual arguments too, though theres an annoying input issue with variances for now, requiring the array of a list input
hardcoded_local_1 = machine.get_local_state(mu=2, sigma= np.array([.75]))
hardcoded_local_2 = machine.get_local_state(mu=3.5, sigma= np.array([1.25]))


fig, ax = plt.subplots(figsize=(10,6))
states = [random, random_local, hardcoded_local_1, hardcoded_local_2]
labels = ['rand','rand_loc', 'local_1', 'local_2']
for s,l in zip(states, labels):
    ax.plot(range(machine.S), s.T, label=l, marker='o')
ax.set_xlabel('state #');
ax.set_ylabel('p(state #)');
fig.legend()


In [ ]:
# and, we can evolve these states according to the rate matrix:
test_state1 = random_local
test_state2 = meps

#here well use the ness as a reference
reference_state = ness
reference_name = 'ness'

#total time T, timestep dt
T = 100
dt = .2


states = [test_state1]
states2 = [test_state2]

for i in range(T):
    states.append(machine.evolve_state(states[-1], dt=dt))
    states2.append(machine.evolve_state(states2[-1], dt=dt))
    

fig, ax = plt.subplots(1,2, figsize=(10,5), sharey=True)

ax[0].hlines(reference_state, xmin=0, xmax=T, colors='k', linestyle='--', label=reference_name)
ax[1].hlines(reference_state, xmin=0, xmax=T, colors='k', linestyle='--')

ax[0].plot(np.squeeze(states))
ax[1].plot(np.squeeze(states2),label=[f'$s_{i}$' for i in range(test_state1.size)])
ax[0].set_xlabel('$t$')
ax[0].set_ylabel('$p_(s_i(t))$')

ax[0].set_title('test_state_1_dynamics');
ax[1].set_title('test_state_2_dynamics');
fig.legend();

In [ ]:
# all of these things also work with ensembles of machines, for example...

# make an ensemble of 100 machines with 50 states each, generated using the default uniform generator for transition rates
#machines = MC(S=50, N=100)

# or whatever generator we want, here is an example of the arrhenious pump
machines = MC(generator=arrhenius_pump_generator, S=20, N=100, n_pumps=30, pump_strength = 6)

In [ ]:
%%time

ness_100 = machines.get_ness()
meps_100 = machines.get_meps()
unif_100 = machines.get_uniform()

ness_epr = machines.get_epr(ness_100)
meps_epr = machines.get_epr(meps_100)
unif_epr = machines.get_epr(unif_100)


ness_activity = machines.get_activity(ness_100)
meps_activity = machines.get_activity(meps_100)
unif_activity = machines.get_activity(unif_100)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14,7))
ax[0].scatter(range(100), unif_epr-meps_epr, label='unif', alpha=.8)
ax[0].scatter(range(100), ness_epr-meps_epr, label='ness', alpha=.8)

ax[1].scatter(range(100), unif_activity)
ax[1].scatter(range(100), ness_activity)
ax[1].scatter(range(100), meps_activity, label='meps')

ax[0].set_title('EPR- MEPR')
ax[1].set_title('activity')

fig.legend()

In [ ]:
# we can look at the distribution of rates too. I dunno why, just for fun I guess

R_in = machines.R > 0
R_out = machines.R <= 0

fig, ax = plt.subplots(1,3, figsize=(15,4))

plot_kwargs = {'linestyle':'None', 'marker':'o', 'markersize':4, 'c':'tab:blue'}
ax[0].plot(meps_epr, machines.R[R_in].reshape(-1,machines.S**2-machines.S), alpha=.1, **plot_kwargs)
ax[1].plot(meps_epr, -machines.R[R_out].reshape(-1,machines.S), alpha=.5, **plot_kwargs)
ax[2].plot(-machines.R[R_out], -machines.R[R_in].reshape(-1,machines.S-1)/machines.R[R_out][:,None], alpha=.025, **plot_kwargs);

for a in ax.ravel():
    a.set_yscale('log')
    a.set_xscale('log')

fig.suptitle('Transition Rates over all Machines')

ax[0].set_xlabel('meps epr')
ax[0].set_ylabel('$R_{i \\neq j}$')

ax[1].set_xlabel('meps epr')
ax[1].set_ylabel('$R_{i=j}$')

ax[2].set_xlabel('$R_{i \\neq j}$')
ax[2].set_ylabel('$R_{i\\neq j} \\quad/ \\quad R_{i = j}$')




In [ ]:
%%time
# example of doing a sweep over many different kinds of generators with different S and N, and saving some of the data.
sizes = [3, 6, 20, 100, 250]
trials = [300, 300, 200, 100, 50]

normal_output, uniform_output, gamma_output, cyclic_output, balanced_output = {}, {}, {}, {}, {}

for a in range(5):
    print(a)
    output = [normal_output, uniform_output, gamma_output, cyclic_output, balanced_output ][a]
    gen = [normal_generator, uniform_generator, gamma_generator, cyclic_generator, detailed_balance_generator][a]
    
    
    for s,trial in zip(SS,trials):
        p = MC(S=s, N=trial, generator=gen)
        #R = [ np.random.uniform(0,1,(S,S)) for i in range(trials)]
        #for p,r in zip(procs,R):
        #    p.set_rate_matrix(r, max_rate=1)
        #print(s, a)   
        dct = {'ness':p.get_epr(p.get_ness()), 
               'meps':p.get_epr(p.get_meps()), 
               'unif':p.get_epr(p.get_uniform()),
               'rand':p.get_epr(p.get_random_state()),
               'local':p.get_epr(p.get_local_state()),
               'N':trial, 'scale':p.scale, 
               'nm_dkl':p.dkl(p.ness,p.meps),'mn_dkl':p.dkl(p.meps,p.ness)}
        
        output[f'{p.S}'] = dct


In [ ]:
# some plotting calls for the above


nstates= [f'{item}' for item in sizes]

# pick the generator
plot_generator = cyclic_output

fig,axs = plt.subplots(1,len(nstates), figsize=(10,6), sharey=True, sharex=True)

# pick which states to plot
state_types = ['local','ness','rand','unif']

for ax, n in zip(axs, nstates):

    for state_type in state_types:
        meps_epr = plot_generator[n]['meps']
        
        ax.scatter(meps_epr, plot_generator[n][state_type]-meps_epr, label=state_type, alpha=.2)

    ax.set_title(f'N={n}')
    
    ax.set_xlabel('MEPS EPR')
    ax.set_ylabel('EPR-MEPS_EPR')

    ax.set_yscale('log')

axs[-1].legend()
axs[0].set_ylim(1E-5,1E2)

In [ ]:
# example of doing a sweep over just the arrhenius pump generator, with different S and N, and saving some of the data.
sizes = [3**(i+1) for i in range(6)]
trials = [300 for i in range(6)]

pump_output = {}

# set pump parameters here
percent = 0.8 
strength = 4
    
for s,trial in zip(sizes,trials):
    print(s)

    num_pumps = max(1, int( percent * ( (s**2-s)/2 ) ) )
    p = MC(S=s, N=trial, generator=arrhenius_pump_generator, pump_strength=strength, n_pumps=num_pumps)
 
    dct = {'ness':p.get_epr(p.get_ness()), 
           'meps':p.get_epr(p.get_meps()), 
           'unif':p.get_epr(p.get_uniform()),
           'rand':p.get_epr(p.get_random_state()),
           'N':trial, 'scale':p.scale, 
           'nm_dkl':p.dkl(p.ness,p.meps),'mn_dkl':p.dkl(p.meps,p.ness)}
    
    pump_output[f'{p.S}'] = dct

In [ ]:
# some example data calls


nstates= [f'{item}' for item in sizes]

plot_generator = pump_output

fig,axs = plt.subplots(1,len(nstates), figsize=(10,6), sharey=True, sharex=True)


state_types = ['ness','rand','unif']

for ax, n in zip(axs, nstates):

    for state_type in state_types:
        meps_epr = plot_generator[n]['meps']

        ax.scatter(meps_epr, plot_generator[n][state_type]-meps_epr, label=state_type, alpha=.2)

    ax.set_title(f'N={n}')
    
    ax.set_xlabel('MEPS EPR')
    ax.set_ylabel('EPR-MEPS_EPR')

    ax.set_yscale('log')

axs[-1].legend()
axs[0].set_ylim(1E-5,1E2)

